In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd /content/drive/MyDrive/yolov5
!pip install -r /content/drive/MyDrive/yolov5/requirements.txt

In [3]:
# 필요한 라이브러리 설치
import os
import shutil
import json
import math
import yaml
import pandas as pd

In [ ]:
#파일 위치
train_images_path = '/content/drive/MyDrive/container_dataset/train_images' # 경로설정
train_labels_path = '/content/drive/MyDrive/container_dataset/train_labels'
valid_images_path = '/content/drive/MyDrive/container_dataset/valid_images'

In [ ]:
# train 폴더 만들어서 .png 파일만 넣기
def copy_files_with_extension(source_folder, destination_folder, file_extension):
        # source_folder 내의 파일 목록 가져오기
        file_list = os.listdir(source_folder)

        for file_name in file_list:
            # 파일의 절대 경로 생성
            file_path = os.path.join(source_folder, file_name)

            # 원하는 파일 확장자 검사
            if file_name.endswith(file_extension):
                # 파일 복사
                shutil.copy(file_path, destination_folder)
                print(f"Copied: {file_name}")

source_folder = train_images_path  # 원본 파일이 있는 폴더 경로
destination_folder = "/content/drive/MyDrive/container_dataset/train"  # 복사될 폴더 경로
file_extension = ".png"  # 복사할 파일의 확장자 (예: .txt, .jpg 등)

# 폴더 생성 (존재하지 않을 경우)
if not os.path.exists(destination_folder):
  os.makedirs(destination_folder)

copy_files_with_extension(source_folder, destination_folder, file_extension)

In [ ]:
# train 폴더 안에 있는 .png 파일 이름과 동일한 label 파일 불러와서 라벨링
destination_folder = "/content/drive/MyDrive/container_dataset/train" # png 파일만 저장된 폴더
png_files = os.listdir(destination_folder)
txt_folder = destination_folder

class_id = 0

for i in range(len(png_files)):
  png_path = os.path.join(destination_folder, png_files[i])
  json_path = os.path.join(train_labels_path, png_files[i])
  json_path = json_path.replace('png', 'json')

  # json 파일 읽기
  with open(json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    for j in range(len(data['features'])):
      imcoords = data['features'][j]['properties']['object_imcoords'].split(',')
      flo_imcoo = []
      for k in range(len(imcoords)):
        imcoords[k] = float(imcoords[k])
      x1,y1, x2,y2, x3,y3, x4,y4 = imcoords
      width = math.sqrt(pow((x2 - x1), 2) + pow((y2 - y1), 2)) # 직사각형 넓이 구하기
      height = math.sqrt(pow((x2 - x3),2) + pow((y2 - y3), 2)) # 직사각형 높이 구하기
      # 직사각형 중심좌표 구하기
      x_center = (x1 + x2 + x3 + x4) / 4
      y_center = (y1 + y2 + y3 + y4) / 4

      # 라벨링 정보 정규화
      # 넓이, 높이 정규화
      norm_width = width / 1024
      norm_height = height / 1024
      # 중심좌표 정규화
      norm_x_center = x_center / 1024
      norm_y_center = y_center / 1024
      txt_path = os.path.join(txt_folder, png_files[i])
      txt_path = txt_path.replace('png', 'txt')
      if j == 0:
        with open(txt_path, 'x') as file:
          file.write(f"{class_id} {norm_x_center} {norm_y_center} {norm_width} {norm_height}\n")
      else:
        with open(txt_path, 'a') as file:
          file.write(f"{class_id} {norm_x_center} {norm_y_center} {norm_width} {norm_height}\n")
print('라벨링 파일 생성 완료')

In [ ]:
# val 폴더 만들어서 .png 파일만 넣기
def copy_files_with_extension(source_folder, destination_folder, file_extension):
        # 폴더 생성 (존재하지 않을 경우)
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        # source_folder 내의 파일 목록 가져오기
        file_list = os.listdir(source_folder)

        for file_name in file_list:
            # 파일의 절대 경로 생성
            file_path = os.path.join(source_folder, file_name)

            # 원하는 파일 확장자 검사
            if file_name.endswith(file_extension):
                # 파일 복사
                shutil.copy(file_path, destination_folder)
        print('완료')

# 예시
source_folder = "/content/drive/MyDrive/container_dataset/valid_images"  # 원본 파일이 있는 폴더 경로
destination_folder = "/content/drive/MyDrive/container_dataset/val"  # 복사될 폴더 경로
file_extension = ".png"  # 복사할 파일의 확장자 (예: .txt, .jpg 등)

copy_files_with_extension(source_folder, destination_folder, file_extension)


In [ ]:
train_folder = '/content/drive/MyDrive/container_dataset/train'
val_folder = '/content/drive/MyDrive/container_dataset/val'

data = {
    'train' : train_folder,
    'val' : val_folder,
    'nc' : 1,
    'names' : ['container']
}

yaml_path = '/content/drive/MyDrive/container_dataset/container.yaml'

with open(yaml_path, 'w') as yaml_file:
  yaml.dump(data, yaml_file)

In [ ]:
os.chdir('/content/drive/MyDrive/yolov5')
# /content/drive/MyDrive/oiltank_dataset/yolov5_5th_weights/weights/best.pt
!python train.py --img 1024 --epochs 20 --batch 16 --data "yaml파일 경로" --cfg "/content/drive/MyDrive/yolov5/models/yolov5s.yaml" --name yolov5_7th --weights "weights 파일 경로"

In [ ]:
os.chdir('/content/drive/MyDrive/yolov5')
# /content/yolov5/runs/train/yolov5_6th/weights/best.pt
!python detect.py --weights "weights 파일 경로" --img 1024 --conf 0.5 --source "#검증할 파일 경로" --save-txt --save-conf

In [ ]:
# yolov5 학습 후에 생성된 weights 폴더 drive에 옮기기

# shutil.rmtree('/content/yolov5/runs/train/yolov5_5th') # 폴더(with 하위파일들) 삭제
shutil.copytree('/content/yolov5/runs/detect/exp', 'container_dataset/detect')

In [ ]:
# txt 파일 csv 파일로 변환
# 열 정보 : 이미지 파일명 앞 8자리/confidence/x1/y1/x2/y2/x3/y3/x4/y4

# txt 파일 읽어서 바운딩 박스 좌표 구하기
txt_folder = '/container_dataset/detect/labels'
txt_list = os.listdir(txt_folder)

# 리스트 선언(데이터프레임 생성을 위한)
file_li =[]
conf_li = []
x1_li = []
y1_li = []
x2_li = []
y2_li = []
x3_li = []
y3_li = []
x4_li = []
y4_li = []

for txt_file in txt_list:
  file_name = txt_file[:8]
  txt_path = os.path.join(txt_folder, txt_file)
  with open(txt_path, 'r') as file:
    lines = file.readlines() # txt 파일 하나 읽기

    for line in lines: # 읽은 파일의 한 줄씩 불러오기
      bb = line.split(' ')
      bb_flo = list(map(float, bb))
      # x_center = bb[1], y_center = bb[2], width = bb[3], height = bb[4], conf = bb[5]
      x_center = bb_flo[1] * 1024
      y_center = bb_flo[2] * 1024
      width = bb_flo[3] * 1024
      height = bb_flo[4] * 1024
      # 좌측 상단 좌표
      x1 = x_center - width / 2
      y1 = y_center + height / 2
      # 우측 하단 좌표
      x3 = x_center + width / 2
      y3 = y_center - height / 2
      # 우측 상단 좌표
      x2 = x3
      y2 = y1
      # 좌측 하단 좌표
      x4 = x1
      y4 = y3
      conf = bb_flo[5]

      # csv 파일로 저장
      # dataframe 만들기
      # 리스트에 현재 바운딩 박스 추가
      file_li.append(file_name)
      conf_li.append(conf)
      x1_li.append(x1)
      y1_li.append(y1)
      x2_li.append(x2)
      y2_li.append(y2)
      x3_li.append(x3)
      y3_li.append(y3)
      x4_li.append(x4)
      y4_li.append(y4)

# 개수 동일
print(len(file_li))
print(len(conf_li))
print(len(x1_li))
print(len(y1_li))
print(len(x2_li))
print(len(y2_li))
print(len(x3_li))
print(len(y3_li))
print(len(x4_li))
print(len(y4_li))

# 데이터프레임 생성
raw_data = {
    'file' : file_li,
    'confidence' : conf_li,
    'x1' : x1_li,
    'y1' : y1_li,
    'x2' : x2_li,
    'y2' : y2_li,
    'x3' : x3_li,
    'y3' : y3_li,
    'x4' : x4_li,
    'y4' : y4_li
}
# 데이터프레임으로 만들기
dt = pd.DataFrame(raw_data)

# 데이터프레임 csv 파일로 만들기
dt.to_csv('/container_dataset/컨테이너n회.csv', index=False)